### Match ECHO videos with PET CFR labels ###
We need this to assign the CFR labels to the videos. The goal is to have a data set with PET CFR values for each video that we will use to train.

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 2000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200227'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200227


#### CFR REPORTS ####

In [3]:
# Rahul's PET studies
pet_echo_file = os.path.join(cfr_data_root, 'bwh_2020-Feb-4_pet_365_diff.txt')
pet_echo_df = pd.read_csv(pet_echo_file, sep = '\t') # Contains all PET MRNs use this one.
pet = pet_echo_df.rename(columns = {'MRN': 'mrn'}).reset_index(drop = True)
print('Unique patients:   {}'.format(len(pet.mrn.unique())))

# Let's rename the columns to allow matching the dates
pet = pet.rename(columns = {'pet_date': 'cfr_study_date',
                            'echo_date': 'rahul_echo_date'})
pet = pet.astype({'cfr_study_date': 'datetime64[ns]',
                  'rahul_echo_date': 'datetime64[ns]'})
rpet = pet.copy()
print(list(rpet.columns))

Unique patients:   3567
['mrn', 'rahul_echo_date', 'cfr_study_date', 'difference(days)']


In [77]:
pet_echo_file = os.path.normpath('/mnt/obi0/phi/pet/pet_cfr/non_defect_plus_p2018.parquet')
pet_echo_df = pd.read_parquet(pet_echo_file)

pet = pet_echo_df.rename(columns = {'pet_date': 'cfr_study_date'})
pet = pet.astype({'cfr_study_date': 'datetime64[ns]'})
#pet = pet.merge(right = rpet, on = ['mrn', 'cfr_study_date'], how='left')
print(pet.shape)

(2659, 112)


In [78]:
print(len(pet.petmrn_identifier.unique()))

2655


In [79]:
# Remove rows or columns with all NAs
pet2 = pet.dropna(how='all')

# Remove rows in pet df with missing values in very important columns
pet2 = pet.dropna(subset=['mrn', 'cfr_study_date', 'unaffected_cfr', 'rest_mbf_unaff'], axis=0)

print(pet2.shape)

(2659, 112)


In [80]:
# NEW PET_REPORT_ID: unique combination of mrn and study date
ID = pet2.groupby(['mrn', 'cfr_study_date']).ngroup()
pet2 = pet2.assign(mrnstudyid=ID)
print('Unique mrn_cfr_date_id values: {}'.format(len(pet2.mrnstudyid.unique())))
print('model_cfr values: {}'.format(len(pet2.unaffected_cfr.unique())))
print('Unique patients: {}'.format(len(pet2.mrn.unique())))

Unique mrn_cfr_date_id values: 2655
model_cfr values: 2650
Unique patients: 2376


In [81]:
# PET CFR REPORTS (OUTDATED)
getStress_reports_file = os.path.join(cfr_data_root, '210_getStressTest.xlsx')
getStress_df = pd.read_excel(getStress_reports_file)
keep_cols = ['reportID', 'mrn', 'subjectid', 'report_number', 'study_date', 'report_date', 'cfr']
getStress_df = getStress_df[keep_cols]

getStress_df = getStress_df.astype({'report_date': 'datetime64[ns]',
                                    'study_date': 'datetime64[ns]'})

pet_cfr = getStress_df.rename(columns = {'report_date': 'cfr_report_date',
                                         'study_date': 'cfr_study_date'})
pet_cfr.head()

,reportID,mrn,subjectid,report_number,cfr_study_date,cfr_report_date,cfr
0,72946,4474599,3,0904-0007,2007-09-28,2007-09-04,1.428232
1,91480,9774688,6,EVS0154093,2008-11-11,2008-11-11,2.015579
2,78543,8880684,7,0802806B,2007-10-31,2008-01-29,2.401606
3,73048,16480857,10,0728923B,2007-11-26,2007-10-29,1.107500
4,100873,7633498,11,EVS0266850,2010-01-29,2010-09-24,1.776545


#### NPY ECHO FILES ####
This is a data frame that contains the filenames, MRNs and dates of all echo studies

In [84]:
echo_files_name = 'echo_npyFiles_BWH_'+cfr_meta_date+'.parquet'
echo = pd.read_parquet(os.path.join(meta_dir, echo_files_name))
echo = echo.assign(echo_study_date = echo.datetime.dt.date,
                   echo_study_time = echo.datetime.dt.time)

echo = echo.astype({'mrn': 'int64',
                    'echo_study_date': 'datetime64[ns]'})

print(echo_files_name)
print('Number of npy files: {}'.format(len(echo.filename.unique())))
print('Unique echo studies: {}'.format(len(echo.study.unique())))
print('Unique patients:     {}'.format(len(echo.mrn.unique())))
print()

# For matching dates with PET, we only need the studies
# So we can greatly reduce this data frame
echo = echo[['study', 'mrn', 'echo_study_date']].drop_duplicates().reset_index(drop = True).\
    sort_values(by = 'mrn', ascending = True)

echo_npyFiles_BWH_200227.parquet
Number of npy files: 1178870
Unique echo studies: 23495
Unique patients:     13378



In [85]:
echo.head()

,study,mrn,echo_study_date
3345,4b7b463ae45ad0e0_4903a585842f436837ee2e0d9dd5,9241,2010-01-26
3350,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,9241,2010-04-02
3425,4b7b463e0550e9de_4903a58051d59d426fe452739a29,41723,2015-11-03
3356,4b7b463fc2f49a36_4903a58dd5465407d53d20ba3790,51599,2018-05-22
3365,4b7b463fc2f49a36_4903a58793b43bb4bb1078d62f4e,51599,2012-03-08


#### PET STUDIES + NPY ECHO DATA ####

In [86]:
pet_echo = pet2.merge(right = echo, on = 'mrn', how = 'left')
pet_echo.head()

,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,unaffected_segments,perf_seg_cfr,perf_segs,rest_global_mbf,var_rest_mbf,stress_global_mbf,var_stress_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,...,cfr_seg_15,cfr_seg_16,cfr_seg_17,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,rest_basal_anterior,rest_basal_anteroseptal,rest_basal_inferoseptal,rest_basal_inferior,rest_basal_inferolateral,rest_basal_anterolateral,rest_mid_anterior,rest_mid_anteroseptal,rest_mid_inferoseptal,rest_mid_inferior,rest_mid_inferolateral,rest_mid_anterolateral,rest_apical_anterior,rest_apical_septal,rest_apical_inferior,rest_apical_lateral,rest_apex,stress_basal_anterior,stress_basal_anteroseptal,stress_basal_inferoseptal,stress_basal_inferior,stress_basal_inferolateral,stress_basal_anterolateral,stress_mid_anterior,stress_mid_anteroseptal,stress_mid_inferoseptal,stress_mid_inferior,stress_mid_inferolateral,stress_mid_anterolateral,stress_apical_anterior,stress_apical_septal,stress_apical_inferior,stress_apical_lateral,stress_apex,height_in,weight_lb,reportID,subjectID,mrnstudyid,study,echo_study_date
0,7924277,2008-08-15,7924277_2008-08-15,0,abnormal,1.368012,1.636577,0.682400,1.116800,5,"[1, 2, 7, 8, 13]",1.202844,"[4, 5, 10, 11, 15]",0.644,0.115952,0.881,0.162315,0.448,0.380,0.435,0.830,0.402,0.334,0.730,0.890,0.537,0.388,0.607,0.823,0.964,0.766,0.445,1.340,1.504,0.691,0.707,0.529,0.348,0.558,0.562,1.325,1.446,0.636,0.502,1.136,1.186,1.415,0.979,0.670,1.174,...,1.505618,0.876119,1.114362,18.0,18.0,0.0,1.09,normal,NaN,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,67.0,133.0,69718.0,2540.0,698,NaN,NaT
1,12853099,2006-01-25,12853099_2006-01-25,0,normal,2.109661,2.124757,1.540882,3.274000,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.000000,[],1.532,0.123958,3.232,0.436719,1.082,0.940,1.135,1.224,1.332,1.945,1.482,1.977,1.475,1.656,1.509,1.444,1.801,2.180,1.677,1.359,1.977,4.291,3.144,2.069,2.657,2.626,3.712,3.981,3.359,2.215,4.010,3.890,3.883,3.249,2.609,3.483,3.443,...,2.076923,2.533481,1.536166,0.0,0.0,0.0,None,not assessed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,160.0,69969.0,3379.0,1084,4a14f25359c230bf_4903a44516dacae1fafd6299fae6,2006-01-24
2,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.011962,1.309667,1.325333,3,"[2, 8, 13]",1.181455,"[4, 5, 6, 10, 11, 15]",1.028,0.086979,1.264,0.092843,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,...,1.725581,1.099682,1.396004,20.0,18.0,2.0,0.97,normal,6602.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,71.0,200.0,69993.0,3542.0,1755,4903a37dd868ef56_4903a44516df7db647ae31be78ef,2006-04-11
3,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.011962,1.309667,1.325333,3,"[2, 8, 13]",1.181455,"[4, 5, 6, 10, 11, 15]",1.028,0.086979,1.264,0.092843,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.3

In [25]:
pet_echo[pet_echo.study=='4b7a857e1fbeb850_4903a58c1bcca937f65d91c9e305']

,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,unaffected_segments,perf_seg_cfr,perf_segs,rest_global_mbf,var_rest_mbf,stress_global_mbf,var_stress_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,...,cfr_seg_17,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,rest_basal_anterior,rest_basal_anteroseptal,rest_basal_inferoseptal,rest_basal_inferior,rest_basal_inferolateral,rest_basal_anterolateral,rest_mid_anterior,rest_mid_anteroseptal,rest_mid_inferoseptal,rest_mid_inferior,rest_mid_inferolateral,rest_mid_anterolateral,rest_apical_anterior,rest_apical_septal,rest_apical_inferior,rest_apical_lateral,rest_apex,stress_basal_anterior,stress_basal_anteroseptal,stress_basal_inferoseptal,stress_basal_inferior,stress_basal_inferolateral,stress_basal_anterolateral,stress_mid_anterior,stress_mid_anteroseptal,stress_mid_inferoseptal,stress_mid_inferior,stress_mid_inferolateral,stress_mid_anterolateral,stress_apical_anterior,stress_apical_septal,stress_apical_inferior,stress_apical_lateral,stress_apex,height_in,weight_lb,reportID,subjectID,rahul_echo_date,difference(days),mrnstudyid,study,echo_study_date
394,1252980,2007-10-01,1252980_2007-10-01,0,normal,1.827052,1.822454,1.920294,3.499647,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.0,[],1.937,0.158027,3.539,0.625532,1.718,1.316,1.082,1.148,1.995,1.796,2.116,2.011,1.797,2.001,2.278,2.192,2.188,2.175,2.366,2.207,2.259,3.144,2.604,1.66,2.277,2.786,3.272,3.912,4.049,3.296,3.698,4.484,4.272,3.818,3.925,4.351,3.791,...,1.839309,0.0,0.0,0.0,0.91,normal,1940.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,132.0,72981.0,1246.0,NaT,NaN,86,4b7a857e1fbeb850_4903a58c1bcca937f65d91c9e305,2019-03-20
8809,1252980,2019-04-02,1252980_2019-04-02,1,normal,1.590000,1.590000,1.030000,1.640000,17,None,NaN,None,1.030,NaN,1.640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,13.0,87,4b7a857e1fbeb850_4903a58c1bcca937f65d91c9e305,2019-03-20


In [87]:
# Filter NAN in study from merged PET report (we don't have .npy files from those studies)
pet_echo_nofiles = pet_echo.loc[pet_echo.study.isnull()].reset_index(drop = True)
petecho = pet_echo.loc[~pet_echo.study.isnull()].reset_index(drop = True)

In [88]:
print('PET with echo videos:')
#print('Number of npy files: {}'.format(len(petecho.filename.unique())))
print('Unique echo studies: {}'.format(len(petecho.study.unique())))
print('Unique patients:     {}'.format(len(petecho.mrn.unique())))

PET with echo videos:
Unique echo studies: 2948
Unique patients:     1662


In [89]:
print('PET without ECHO:')
#print('Number of npy files: {}'.format(len(petecho.filename.unique())))
print('Unique CFR studies: {}'.format(len(pet_echo_nofiles.reportID.unique())))
print('Unique patients:     {}'.format(len(pet_echo_nofiles.mrn.unique())))

PET without ECHO:
Unique CFR studies: 771
Unique patients:     714


### Match ECHO study with PET reportID  ###
Goal:
Find a CFR label for each echo video taken from the closest PET measurement.
Requirement: We want to pick the most recent CFR measurments for each echo that is not older than 365 days.
For this, we need a data frame that contains for a given MRN:

- All echo studies for this patient
- A date for each echo study
- All pet reports for that patient
- Date for each pet report

In [90]:
petecho.head()

,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,unaffected_segments,perf_seg_cfr,perf_segs,rest_global_mbf,var_rest_mbf,stress_global_mbf,var_stress_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,...,cfr_seg_15,cfr_seg_16,cfr_seg_17,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,rest_basal_anterior,rest_basal_anteroseptal,rest_basal_inferoseptal,rest_basal_inferior,rest_basal_inferolateral,rest_basal_anterolateral,rest_mid_anterior,rest_mid_anteroseptal,rest_mid_inferoseptal,rest_mid_inferior,rest_mid_inferolateral,rest_mid_anterolateral,rest_apical_anterior,rest_apical_septal,rest_apical_inferior,rest_apical_lateral,rest_apex,stress_basal_anterior,stress_basal_anteroseptal,stress_basal_inferoseptal,stress_basal_inferior,stress_basal_inferolateral,stress_basal_anterolateral,stress_mid_anterior,stress_mid_anteroseptal,stress_mid_inferoseptal,stress_mid_inferior,stress_mid_inferolateral,stress_mid_anterolateral,stress_apical_anterior,stress_apical_septal,stress_apical_inferior,stress_apical_lateral,stress_apex,height_in,weight_lb,reportID,subjectID,mrnstudyid,study,echo_study_date
0,12853099,2006-01-25,12853099_2006-01-25,0,normal,2.109661,2.124757,1.540882,3.274000,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.000000,[],1.532,0.123958,3.232,0.436719,1.082,0.940,1.135,1.224,1.332,1.945,1.482,1.977,1.475,1.656,1.509,1.444,1.801,2.180,1.677,1.359,1.977,4.291,3.144,2.069,2.657,2.626,3.712,3.981,3.359,2.215,4.010,3.890,3.883,3.249,2.609,3.483,3.443,...,2.076923,2.533481,1.536166,0.0,0.0,0.0,None,not assessed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,160.0,69969.0,3379.0,1084,4a14f25359c230bf_4903a44516dacae1fafd6299fae6,2006-01-24
1,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.011962,1.309667,1.325333,3,"[2, 8, 13]",1.181455,"[4, 5, 6, 10, 11, 15]",1.028,0.086979,1.264,0.092843,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,...,1.725581,1.099682,1.396004,20.0,18.0,2.0,0.97,normal,6602.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,71.0,200.0,69993.0,3542.0,1755,4903a37dd868ef56_4903a44516df7db647ae31be78ef,2006-04-11
2,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.011962,1.309667,1.325333,3,"[2, 8, 13]",1.181455,"[4, 5, 6, 10, 11, 15]",1.028,0.086979,1.264,0.092843,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,...,1.725581,1.099682,1.396004,20.0,18.0,2.0,0.97,normal,6602.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,71.0,200.0,69993.0,3542.0,1755,4903a37dd868ef56_4903a44516d8ca53ad30d0e53fe4,2006-03-06
3,12627030,2006-02-10,12627030_2006-02-10,0,abnormal,1.588915,1.248541,2.056000,2.567000,1,[14],1.708358,"[1, 3, 5, 12, 16]",1.732,0.103225,2.752,1.735979,1.984,1.499,1.556,1.333,1.134,1.292,2.499,1.883,1.703,1.794,1.873,1.933,1.767,2.056,1.662,1.804,1.754,2.940,0.664,0.640,2.197

In [32]:
# Pick a patient
#np.random.seed(112)
#mrn = np.random.choice(petecho.mrn.unique(), size = 1,)[0]
mrn = 15467822
print('MRN {}'.format(mrn))
petecho_mrn = petecho[petecho.mrn == mrn].reset_index(drop=True)
print(petecho_mrn[['mrn', 'mrnstudyid', 'cfr_study_date', 'echo_study_date', 'study', 'unaffected_cfr']].iloc[8:12])

MRN 15467822
        mrn  mrnstudyid cfr_study_date echo_study_date                                          study  unaffected_cfr
8  15467822        1292     2012-10-11      2014-10-28  4a1347300a20f7e7_4903a581a681209d89d1fed6737d        1.366817
9  15467822        1292     2012-10-11      2012-09-06  4a1347300a20f7e7_4903a58793be6eaf2db65cba1044        1.366817


In [33]:
petecho.columns

Index(['mrn', 'cfr_study_date', 'petmrn_identifier', 'post-2018', 'myocardial_perfusion', 'global_cfr_calc', 'unaffected_cfr', 'rest_mbf_unaff', 'stress_mbf_unaff', 'num_unaffected_segs',
       ...
       'stress_apex', 'height_in', 'weight_lb', 'reportID', 'subjectID', 'rahul_echo_date', 'difference(days)', 'mrnstudyid', 'study', 'echo_study_date'], dtype='object', length=117)

In [36]:
# Pick a study
echo_study = petecho_mrn.study.unique()[2]
echo_study_date = petecho_mrn[petecho_mrn.study == echo_study].echo_study_date.values[0]
print(echo_study)
print(echo_study_date)

# Now, list the reportIDs with the corresponding dates
pet_dates = petecho_mrn[['mrnstudyid', 'cfr_study_date']].groupby('mrnstudyid').first().reset_index(drop=False)
pet_dates = pet_dates.assign(days=(pet_dates.cfr_study_date-echo_study_date).dt.days)
pet_dates = pet_dates.assign(days_abs = np.abs(pet_dates.days))
print()
print(pet_dates)

# Now, we can find the earliest pet_study_date and the corresponding reportID
ser = pet_dates.loc[pet_dates.days_abs.idxmin()]
print('ReportID of earliest PET study {}'.format(ser.mrnstudyid))
print('cfr_days_after_echo {}'.format(ser.days))

4a1347300a20f7e7_4903a581a723bbe47f2a139a23e4
2014-08-29T00:00:00.000000000

   mrnstudyid cfr_study_date  days  days_abs
0        1292     2012-10-11  -687       687
ReportID of earliest PET study 1292
cfr_days_after_echo -687


In [91]:
# Now, we can come up with a function that determines the reportID and the dates of the earliest CFR
# For any echo date for a given patient.

def get_earliest_cfr_reports(df_cfr_echo):
    """ 
    Determine the earliest CFR measurement for all studies
    args: df_cfr_echo: pd.DataFrame() 
          with columns: mrn, study, reportID, cfr_study_date, echo_study_date
    returns: cfr_echo_days: pd.DataFrame()
          with columns: mrn, study, reportID, cfr_days_after_echo
    """

    # Loop through all MRNs in df_pet_echo
    mrn_list = sorted(list(df_cfr_echo.mrn.unique()))
    cfr_echo_days = pd.DataFrame()
    for m, mrn in enumerate(mrn_list):

        if (m+1)%200==0:
            print('Processing MRN {} of {}'.format(m+1, len(mrn_list)))

        df_mrn = df_cfr_echo[df_cfr_echo.mrn == mrn]
        echo_study_list = list(df_mrn.study.unique())

        # Loop through all studies for this MRN
        for study in echo_study_list:
            echo_study_date = df_mrn[df_mrn.study == study].echo_study_date.values[0]

            # Find all pet dates for this patient and add the difference to the echo dates
            pet_dates = df_mrn[['mrnstudyid', 'cfr_study_date']].groupby('mrnstudyid').first().reset_index(drop=False)
            pet_dates = pet_dates.assign(days_post_cfr=(echo_study_date-pet_dates.cfr_study_date).dt.days)
            pet_dates = pet_dates.assign(days_abs = np.abs(pet_dates.days_post_cfr))
            ser = pet_dates.loc[pet_dates.days_abs.idxmin()]

            # We can return the reportID and the days of the earliest pet study
            study_dict = {'mrn': [mrn],
                          'study': [study],
                          'echo_study_date': [echo_study_date],
                          'mrnstudyid': [ser.mrnstudyid],
                          'days_post_cfr': [ser.days_post_cfr]}
            cfr_echo_days = pd.concat([cfr_echo_days, pd.DataFrame(study_dict)], ignore_index=True)
    
    return cfr_echo_days

In [92]:
cfr_echo_days = get_earliest_cfr_reports(petecho)

Processing MRN 200 of 1662
Processing MRN 400 of 1662
Processing MRN 600 of 1662
Processing MRN 800 of 1662
Processing MRN 1000 of 1662
Processing MRN 1200 of 1662
Processing MRN 1400 of 1662
Processing MRN 1600 of 1662


In [93]:
cfr_echo_days.dtypes

mrn                         int64
study                      object
echo_study_date    datetime64[ns]
mrnstudyid                  int64
days_post_cfr               int64
dtype: object

In [94]:
pet2.head()

,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,unaffected_segments,perf_seg_cfr,perf_segs,rest_global_mbf,var_rest_mbf,stress_global_mbf,var_stress_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,...,cfr_seg_13,cfr_seg_14,cfr_seg_15,cfr_seg_16,cfr_seg_17,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,rest_basal_anterior,rest_basal_anteroseptal,rest_basal_inferoseptal,rest_basal_inferior,rest_basal_inferolateral,rest_basal_anterolateral,rest_mid_anterior,rest_mid_anteroseptal,rest_mid_inferoseptal,rest_mid_inferior,rest_mid_inferolateral,rest_mid_anterolateral,rest_apical_anterior,rest_apical_septal,rest_apical_inferior,rest_apical_lateral,rest_apex,stress_basal_anterior,stress_basal_anteroseptal,stress_basal_inferoseptal,stress_basal_inferior,stress_basal_inferolateral,stress_basal_anterolateral,stress_mid_anterior,stress_mid_anteroseptal,stress_mid_inferoseptal,stress_mid_inferior,stress_mid_inferolateral,stress_mid_anterolateral,stress_apical_anterior,stress_apical_septal,stress_apical_inferior,stress_apical_lateral,stress_apex,height_in,weight_lb,reportID,subjectID,mrnstudyid
0,7924277,2008-08-15,7924277_2008-08-15,0,abnormal,1.368012,1.636577,0.682400,1.116800,5,"[1, 2, 7, 8, 13]",1.202844,"[4, 5, 10, 11, 15]",0.644,0.115952,0.881,0.162315,0.448,0.380,0.435,0.830,0.402,0.334,0.730,0.890,0.537,0.388,0.607,0.823,0.964,0.766,0.445,1.340,1.504,0.691,0.707,0.529,0.348,0.558,0.562,1.325,1.446,0.636,0.502,1.136,1.186,1.415,0.979,0.670,1.174,...,1.467842,1.278068,1.505618,0.876119,1.114362,18.0,18.0,0.0,1.09,normal,NaN,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,67.0,133.0,69718.0,2540.0,698
1,12853099,2006-01-25,12853099_2006-01-25,0,normal,2.109661,2.124757,1.540882,3.274000,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.000000,[],1.532,0.123958,3.232,0.436719,1.082,0.940,1.135,1.224,1.332,1.945,1.482,1.977,1.475,1.656,1.509,1.444,1.801,2.180,1.677,1.359,1.977,4.291,3.144,2.069,2.657,2.626,3.712,3.981,3.359,2.215,4.010,3.890,3.883,3.249,2.609,3.483,3.443,...,1.803998,1.196789,2.076923,2.533481,1.536166,0.0,0.0,0.0,None,not assessed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,160.0,69969.0,3379.0,1084
2,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.011962,1.309667,1.325333,3,"[2, 8, 13]",1.181455,"[4, 5, 6, 10, 11, 15]",1.028,0.086979,1.264,0.092843,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,...,1.110870,1.528986,1.725581,1.099682,1.396004,20.0,18.0,2.0,0.97,normal,6602.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,71.0,200.0,69993.0,3542.0,1755
3,12627030,2006-02-10,12627030_2006-02-10,0,abnormal,1.588915,1.248541,2.056000,2.567000,1,[14],1.708358,"[1, 3, 5, 12, 16]",1.732,0.103225,2.752,1.735979,1.984,1.499,1.556,1.333,1.134,1.292,2.499,1.883,1.703,1.794,1.873,1.933,1.767,2.056,1.662,1.804,1.754,2.940,0.664,0.640,2.197,1.413,3.395,4.533,1.301,2.149,2.391,3.300,4.714,3.184,2.567,4.154,4.662,...,1.801924,1.248541,2.499398,2.584257,2.04960

In [95]:
# We can merge this back with the original PET reports
pet_cfr_echo_days = cfr_echo_days.merge(right = pet2, on = ['mrnstudyid', 'mrn'], how = 'left')

In [96]:
# Next, we will filter those echo studies that occurred earlier or later than 365 days after CFR
pet_cfr_echo_days = pet_cfr_echo_days.assign(abs_cfr_days = np.abs(pet_cfr_echo_days.days_post_cfr))
pet_cfr_echo_days = pet_cfr_echo_days[pet_cfr_echo_days.abs_cfr_days<=365].drop(columns = ['abs_cfr_days'])
# Sort this by study so that we know how well we matched the dates
pet_cfr_echo_days = pet_cfr_echo_days.sort_values(by = 'study')
pet_cfr_echo_days.head(4)

,mrn,study,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,unaffected_segments,perf_seg_cfr,perf_segs,rest_global_mbf,var_rest_mbf,stress_global_mbf,var_stress_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,...,cfr_seg_12,cfr_seg_13,cfr_seg_14,cfr_seg_15,cfr_seg_16,cfr_seg_17,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,rest_basal_anterior,rest_basal_anteroseptal,rest_basal_inferoseptal,rest_basal_inferior,rest_basal_inferolateral,rest_basal_anterolateral,rest_mid_anterior,rest_mid_anteroseptal,rest_mid_inferoseptal,rest_mid_inferior,rest_mid_inferolateral,rest_mid_anterolateral,rest_apical_anterior,rest_apical_septal,rest_apical_inferior,rest_apical_lateral,rest_apex,stress_basal_anterior,stress_basal_anteroseptal,stress_basal_inferoseptal,stress_basal_inferior,stress_basal_inferolateral,stress_basal_anterolateral,stress_mid_anterior,stress_mid_anteroseptal,stress_mid_inferoseptal,stress_mid_inferior,stress_mid_inferolateral,stress_mid_anterolateral,stress_apical_anterior,stress_apical_septal,stress_apical_inferior,stress_apical_lateral,stress_apex,height_in,weight_lb,reportID,subjectID
2915,35133560,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,2018-11-02,2618,191,2018-04-25,35133560_2018-04-25,0,normal,1.716143,1.699871,0.934766,1.588982,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.000000,[],0.923,0.008917,1.584,0.038417,0.762083,0.915500,0.904000,0.889458,0.902542,0.843125,0.907667,1.070250,1.033500,1.047292,1.142375,0.943542,0.876583,0.965445,0.951972,0.897722,0.837964,1.191375,1.306250,1.355375,1.418083,1.704375,1.709792,1.462000,1.760792,1.744250,1.675042,1.886167,1.629292,...,1.726783,1.733784,1.870440,1.876251,1.704592,1.823339,0.0,0.0,0.0,None,normal,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,186.0,162094.0,7817.0
2913,35014216,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,2019-04-25,2616,352,2018-05-08,35014216_2018-05-08,0,abnormal,2.644905,2.876778,0.636890,1.832192,8,"[1, 6, 7, 12, 13, 14, 16, 17]",1.651081,"[3, 4, 10]",0.628,0.006229,1.661,0.123543,0.724250,0.685042,0.641083,0.641250,0.732292,0.722583,0.775958,0.730625,0.606458,0.639917,0.733583,0.622375,0.627306,0.542694,0.556417,0.582528,0.497429,1.978625,1.615375,1.414708,0.711083,1.410500,1.884083,1.847375,1.994458,1.829250,1.048000,1.774958,1.922750,...,3.089375,2.707036,3.617701,3.260995,3.157073,3.064116,9.0,0.0,9.0,None,normal,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,155.0,162416.0,7843.0
2914,35029545,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,2017-05-16,2617,-69,2017-07-24,35029545_2017-07-24,0,abnormal,2.698341,2.554037,0.599508,1.531167,5,"[1, 5, 6, 7, 12]",2.957351,"[9, 14, 15]",0.663,0.013291,1.789,0.122294,0.632167,0.781583,0.799250,0.603167,0.554042,0.566417,0.697708,0.881542,0.909667,0.710167,0.563667,0.547208,0.775917,0.728250,0.667417,0.568194,0.641536,1.601500,1.912792,2.313833,1.982792,1.652250,1.309667,1.614875,2.217625,2.516625,1.980917,1.633958,1.477542,...,2.700145,1.987649,2.848762,3.335913,2.583818,2.491400,7.0,5.0,2.0,None,normal,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,64.0,143.0,156019.0,7376.0
2920,35583509,48b09

In [97]:
# To match with filenames, we really only need one study per row. 
# Remove colums where some rows have missing values
pet_cfr_echo_days_noNA = pet_cfr_echo_days.dropna(axis=1, how='any') 
pet_cfr_echo_days_study = pet_cfr_echo_days_noNA.groupby('study').first().reset_index(drop=False)

print('Original shape: {} now: {}'.format(pet_cfr_echo_days.shape,
                                          pet_cfr_echo_days_study.shape))

print('Original number of studies: {} now: {}'.format(len(cfr_echo_days.study.unique()),
                                                      len(pet_cfr_echo_days_study.study.unique())))

Original shape: (2296, 116) now: (2291, 16)
Original number of studies: 2948 now: 2291


In [98]:
d = pet_cfr_echo_days_study[['mrn', 'study', 'cfr_study_date', 'echo_study_date', 'days_post_cfr']]
d.head(10)

,mrn,study,cfr_study_date,echo_study_date,days_post_cfr
0,35133560,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,2018-04-25,2018-11-02,191
1,35014216,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,2018-05-08,2019-04-25,352
2,35029545,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,2017-07-24,2017-05-16,-69
3,35583509,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,2018-08-08,2017-09-06,-336
4,35401116,48b095533cbfdf01_4903a58c1bcb2198682a98b7503a,2019-04-11,2019-04-09,-2
5,35401116,48b095533cbfdf01_4903a58dd5441181fcd696f7d50c,2019-04-11,2018-07-25,-260
6,35647072,48b097cb6874ce59_4903a582edf18da17812c4897c25,2017-11-01,2017-10-25,-7
7,35995307,48b0984d74745d55_4903a582edf18da3b9aa6d4547eb,2017-10-27,2017-10-27,0
8,34940536,48b1a5aeaf91d4d2_4903a582edf18da48d51c09a959b,2017-06-26,2017-10-20,116
9,34530097,48b1a9c06ce2b304_4903a582ec7226d2e7bd522f25ce,2017-04-18,2017-03-31,-18


In [99]:
pet_cfr_echo_days_study.head(2)

,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,rest_global_mbf,stress_global_mbf
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2618,191,2018-04-25,35133560_2018-04-25,0,normal,1.716143,1.699871,0.934766,1.588982,17,0.923,1.584
1,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2616,352,2018-05-08,35014216_2018-05-08,0,abnormal,2.644905,2.876778,0.636890,1.832192,8,0.628,1.661


In [100]:
print(pet_cfr_echo_days_study.shape)
pet_cfr_echo_days_study.head()

(2291, 16)


,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,rest_global_mbf,stress_global_mbf
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2618,191,2018-04-25,35133560_2018-04-25,0,normal,1.716143,1.699871,0.934766,1.588982,17,0.923,1.584
1,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2616,352,2018-05-08,35014216_2018-05-08,0,abnormal,2.644905,2.876778,0.636890,1.832192,8,0.628,1.661
2,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,35029545,2017-05-16,2617,-69,2017-07-24,35029545_2017-07-24,0,abnormal,2.698341,2.554037,0.599508,1.531167,5,0.663,1.789
3,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,35583509,2017-09-06,2620,-336,2018-08-08,35583509_2018-08-08,0,abnormal,1.994660,2.026460,0.544319,1.103042,3,0.749,1.494
4,48b095533cbfdf01_4903a58c1bcb2198682a98b7503a,35401116,2019-04-09,2619,-2,2019-04-11,35401116_2019-04-11,1,normal,1.110000,1.110000,0.790000,0.880000,17,0.790,0.880


In [101]:
print('Number of echo studies within 365 days of cfr {}'.format(len(pet_cfr_echo_days_study.study.unique())))
print('Total number of studies in petecho df {}'.format(len(petecho.study.unique())))
print('Unique patients:     {}'.format(len(pet_cfr_echo_days_study.mrn.unique())))

Number of echo studies within 365 days of cfr 2291
Total number of studies in petecho df 2948
Unique patients:     1577


In [102]:
len(pet_cfr_echo_days_study.mrnstudyid.unique())

1638

In [69]:
# We can now save the data frame with the echo studies and the matched CFR measurements
match_filename = 'pet_match365_diff_'+cfr_meta_date+'.parquet'
pet_cfr_echo_days_study.to_parquet(os.path.join(meta_dir, match_filename))

# Save also as .csv file
match_filename_csv = 'pet_match365_diff_'+cfr_meta_date+'.csv'
pet_cfr_echo_days_study.to_csv(os.path.join(meta_dir, match_filename_csv))

In [70]:
pet_cfr_echo_days_study.head()

,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,unaffected_cfr,rest_mbf_unaff,stress_mbf_unaff,num_unaffected_segs,rest_global_mbf,stress_global_mbf
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2618,191,2018-04-25,35133560_2018-04-25,0,normal,1.716143,1.699871,0.934766,1.588982,17,0.923,1.584
1,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2616,352,2018-05-08,35014216_2018-05-08,0,abnormal,2.644905,2.876778,0.636890,1.832192,8,0.628,1.661
2,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,35029545,2017-05-16,2617,-69,2017-07-24,35029545_2017-07-24,0,abnormal,2.698341,2.554037,0.599508,1.531167,5,0.663,1.789
3,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,35583509,2017-09-06,2620,-336,2018-08-08,35583509_2018-08-08,0,abnormal,1.994660,2.026460,0.544319,1.103042,3,0.749,1.494
4,48b095533cbfdf01_4903a58c1bcb2198682a98b7503a,35401116,2019-04-09,2619,-2,2019-04-11,35401116_2019-04-11,1,normal,1.110000,1.110000,0.790000,0.880000,17,0.790,0.880


#### Filter the large echo_BWH_npy_feather_files dataframe for easier metadata collection ####

In [71]:
# With the names of the matched echo studies, we can now get the file names
# of the meta files that we will need

meta_files_name = 'echo_BWH_npy_feather_files_'+cfr_meta_date+'.parquet'
meta_files_cfr_name = 'echo_BWH_npy_feather_files_cfr_'+cfr_meta_date+'.parquet'

meta_files_df = pd.read_parquet(os.path.join(meta_dir, meta_files_name))
meta_files_df_matched = meta_files_df[meta_files_df.study.isin(pet_cfr_echo_days_study.study.unique())]
print(meta_files_df_matched.shape)
print('Unique studies in meta data: {}'.format(len(meta_files_df_matched.study.unique())))
meta_files_df_matched.head(2)
meta_files_df_matched.to_parquet(os.path.join(meta_dir, meta_files_cfr_name))

(602084, 8)
Unique studies in meta data: 2291
